In [ ]:
# pip install bs4
# pip install selenium
# pip install requests
# pip install webdriver-manager

https://medium.com/analytics-vidhya/web-scraping-google-search-results-with-selenium-and-beautifulsoup-4c534817ad88

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
# Import the beautifulsoup and request libraries of python.
import requests
import bs4

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

/var/folders/m5/tw1ymsrj4jjg5vcyxyqy8r280000gn/T/ipykernel_72354/1014324415.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
/var/folders/m5/tw1ymsrj4jjg5vcyxyqy8r280000gn/T/ipykernel_72354/1014324415.py:3: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)


In [3]:
## Google search
## blocked due to high frequency search

from time import sleep


text= "innovation"
domain = "site%3Aupenn.edu+"
links = [] # Initiate empty list to capture final results
# Specify number of pages on google search, each page contains 10 #links

n_pages = 10
for page in range(1, n_pages):
    url = "https://www.google.edu/search?" + domain + text + "&start=" + str((page - 1) * 10)
    driver.get(url)
    soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')
    # soup = BeautifulSoup(r.text, 'html.parser')
    print(soup)
    search = soup.find_all('div', class_="yuRUbf")
    print(search)
    for h in search:
        links.append(h.a.get('href'))
    time.sleep(3)

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: headless chrome=105.0.5195.125)
Stacktrace:
0   chromedriver                        0x000000010aedd788 chromedriver + 4515720
1   chromedriver                        0x000000010ae619d3 chromedriver + 4008403
2   chromedriver                        0x000000010aaf412a chromedriver + 413994
3   chromedriver                        0x000000010aaedd8e chromedriver + 388494
4   chromedriver                        0x000000010aae14ad chromedriver + 337069
5   chromedriver                        0x000000010aae2370 chromedriver + 340848
6   chromedriver                        0x000000010aae1718 chromedriver + 337688
7   chromedriver                        0x000000010aae0bd4 chromedriver + 334804
8   chromedriver                        0x000000010aadfa87 chromedriver + 330375
9   chromedriver                        0x000000010aadfe02 chromedriver + 331266
10  chromedriver                        0x000000010aaf5d

In [ ]:
links

In [4]:
import urllib.request
# Penn Website search https://www.upenn.edu/search
text= "innovation"
links = [] # Initiate empty list to capture final results
# Specify number of pages on google search, each page contains 10 #links

n_pages = 5
for page in range(1, n_pages):
   #https://www.upenn.edu/search?as_q=innovation#gsc.tab=0&gsc.q=innovation&gsc.page=1 
    url = "https://www.upenn.edu/search?as_q=" + text + "#gsc.tab=0&gsc.q=innovation&gsc.page=" + str((page))
    driver.get(url)
    soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')
    search = soup.find_all('div', class_="gsc-thumbnail-inside")
    for h in search:
        link = h.a.get('href')
        title = h.a.text
        links.append(link)
        try:
            urllib.request.urlretrieve(link, 'html/' + title + '.html')
        except:
            continue
        print("Downloading "+title)

In [ ]:
  # Make two strings with default google search URL
# 'https://google.com/search?q=' and
# our customized search keyword.
# Concatenate them
text= "innovation"
domain = "site%3Aupenn.edu+"
url = 'https://google.com/search?q=' + domain + text
  
# Fetch the URL data using requests.get(url),
# store it in a variable, request_result.
request_result=requests.get( url )
  
# Creating soup from the fetched request
soup = bs4.BeautifulSoup(request_result.text,
                         "html.parser")
print(soup)

In [ ]:
# soup.find.all( h3 ) to grab 
# all major headings of our search result,
heading_object=soup.find_all( 'h3' )
  
# Iterate through the object 
# and print it as a string.
for info in heading_object:
    print(info.getText())
    print("------")

In [ ]:
import requests
import bs4
  
# Make two strings with default google search URL
# 'https://google.com/search?q=' and
# our customized search keyword.
# Concatenate them
text= "geeksforgeeks"
url = 'https://google.com/search?q=' + text
  
# Fetch the URL data using requests.get(url),
# store it in a variable, request_result.
request_result=requests.get( url )
  
# Creating soup from the fetched request
soup = bs4.BeautifulSoup(request_result.text,
                         "html.parser")
print(soup)
